In [ ]:
googleColab = True

if googleColab:
  from google.colab import drive
  drive.mount('/content/drive')
  # CHANGE THIS PATH IF USING COLAB
  %cd "/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Applied Artificial Intelligence/Assignment"

Mounted at /content/drive
/content/drive/MyDrive/Msc Artificial Intelligence/Semester 1/Applied Artificial Intelligence/Assignment


In [ ]:
import pandas as pd
import numpy as np
import nltk
import xgboost
import sklearn
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textPreProcessing import DataProcessor

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Downloads dataset into pandas dataframe
dataframe = pd.read_csv("./Dataset/training.csv",
                        encoding='ISO-8859-1',
                        names=['target','ids','date','flag','user','tweet'])

In [ ]:
# Changes the target value for postive from 4 to 1 so it is more compatible with binary classification
dataframe['target'] = np.where(dataframe['target'] == 4, 1, 0)

In [ ]:
# Loads the training and test data used by the classifier model
indexes = np.load("./datasetBothModels/indexes.npy")
trainDataframe = dataframe.loc[indexes]
testIndexes = np.load("./datasetBothModels/testIndexes.npy")
testDataframe = dataframe.loc[testIndexes]

In [ ]:
# Cleans data and converts it to list
cleaner = DataProcessor()

xTrain = cleaner.CleanTextData(trainDataframe["tweet"]).values.tolist()
yTrain = trainDataframe["target"].values.tolist()

xTest = cleaner.CleanTextData(testDataframe["tweet"]).values.tolist()
yTest = testDataframe["target"].values.tolist()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# Vectorizes the words and converts to xgboost compatible matrices
vectorizer = TfidfVectorizer(max_features=5000)

xTrain = vectorizer.fit_transform(xTrain)
xTrain = xTrain.toarray()

xTest = vectorizer.fit_transform(xTest)
xTest = xTest.toarray()

xTrain = xgboost.DMatrix(xTrain)
xTest = xgboost.DMatrix(xTest)
yTrain = xgboost.DMatrix(np.reshape(yTrain, (100000, 1)))
yTest = xgboost.DMatrix(np.reshape(yTest, (9375, 1)))

In [ ]:
# Trains classifier on the data
dataLength = 100000

xgModel = xgboost.XGBClassifier(max_leaves=0, max_depth=6, n_estimators=1000, learning_rate=0.01, objective='binary:logistic')
xgModel.fit(xTrain[:dataLength], yTrain[:dataLength])

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=0,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# predicts test data values
predictions = xgModel.predict(xTest[:dataLength])

In [ ]:
# Calculates accuracy on test data
correct = 0
for i, pred in enumerate(predictions):
  if pred == yTest[i]:
    correct += 1

print(correct / len(predictions))

0.49824


In [ ]:
# Saves model
xgModel.save_model("./models/xgBoostModel4N.txt")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:23:55] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [ ]:
# Loads and evaluates model on test dataset
xgModel = xgboost.Booster()
xgModel.load_model("./models/xgBoostModel4N.txt")

predictions = xgModel.predict(xTest)

clampedPredictions = np.where(predictions > 0.5, predictions, 0)
clampedPredictions = np.where(clampedPredictions <= 0.5, clampedPredictions, 1)

labels = yTest.get_data().data

correct = 0
for i, pred in enumerate(clampedPredictions):
  if pred == labels[i]:
    correct += 1

print(correct / len(predictions))